In [1]:
from federated_learning.utils import SHAPUtil
from federated_learning import ClientPlane, Configuration

In [2]:
class ObserverConfiguration():
    experiment_type = "datasize_shap"
    experiment_id = 0
    test = True
    dataset_type = "MNIST"
    
    # Client Configurations 
    client_name = "client"
    client_type = "client"

In [3]:
config = Configuration()
data = config.DATASET(config)
shap_util = SHAPUtil(data.test_dataloader)
observer_config = ObserverConfiguration()
config.NUMBER_OF_CLIENTS = 200
observer_config.experiment_id = 0
client_plane = ClientPlane(config, observer_config, data, shap_util)
client_plane.random_client_ids()
client_plane.poison_clients()
client_plane.clients[client_plane.poisoned_clients[0]].observer.poisoned

MNIST training data loaded.
MNIST test data loaded.
Create 200 clients with dataset of size 300
Flip 100.0% of the 5 labels to 4
20/50 clients poisoned
40/50 clients poisoned


In [ ]:
client_plane.clients[0].test()
for epoch in range(1, config.N_EPOCHS + 1):
    client_plane.clients[0].train(epoch)
    client_plane.clients[0].test()
client_plane.clients[0].push_metrics()